In [19]:
# !pip install aiomoex
# !pip install tinkoff-investments

Открытые вопросы:
* Стоит ли описывать варианты получения данных и причину выбора?

In [ ]:
from tinkoff.invest import AsyncClient, CandleInterval
from tinkoff.invest.caching.instruments_cache.instruments_cache import InstrumentsCache
from tinkoff.invest.caching.instruments_cache.settings import InstrumentsCacheSettings

import pandas as pd
from dotenv import load_dotenv

import os
import asyncio
from datetime import datetime, timezone, timedelta
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

In [21]:
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

In [ ]:
load_dotenv()

TOKEN = "t.eRZ0sm51xUBVmE4xcdx4cebt0iTp8oIbMUFhdf2Ot7GOG7v0BjTryycaTnwiuyJbwpiqpLnKK6WNMCSdHiPZlg"
# os.getenv("TOKEN")

Получить FIGI токена по тикеру

FIGI - Financial Instrument Global Identifier

In [ ]:
def ticker_to_figi(ticker: str, token: str = TOKEN) -> str:
    with Client(token) as client:
        results = client.instruments.find_instrument(query=ticker).instruments
        for item in results:
            if item.ticker.upper() == ticker.upper():
                print(f"{item.ticker} → {item.name[:40]:10} | {item.instrument_type:2} | {item.figi}")
                return item.figi
        
        # Если точного совпадения нет — возвращаем первый найденный (обычно правильный)
        if results:
            best = results[0]
            print(f"Точного совпадения нет, ближайший: {best.ticker} → {best.name[:10]:10} | {best.instrument_type:2} | {best.figi}")
            return best.figi
        
        raise ValueError(f"Ничего не найдено: '{ticker}'. Проверь тикер.")

print(ticker_to_figi("SBER"))

SBER → Сбер Банк  | share | TCS704730N88
TCS704730N88


In [ ]:
instruments_cache = None

async def init_cache():
    global instruments_cache
    async with AsyncClient(TOKEN) as client:
        settings = InstrumentsCacheSettings()
        instruments_cache = InstrumentsCache(
            settings=settings,
            instruments_service=client.instruments
        )
        # Принудительно заполняем кэш
        await instruments_cache.shares()
        await instruments_cache.etfs()
        await instruments_cache.indices()
        print("Кэш инструментов загружен (shares, etfs, indices)")

#")

def get_figi(ticker: str) -> str:
    """Надёжно получает BBG-FIGI через официальный кэш"""
    if instruments_cache is None:
        raise RuntimeError("Сначала вызови await init_cache()")
    
    ticker = ticker.upper()
    
    # Проверяем индексы
    for instr in instruments_cache.indices().instruments:
        if instr.ticker.upper() == ticker:
            return instr.figi
    
    # Проверяем акции
    for instr in instruments_cache.shares().instruments:
        if instr.ticker.upper() == ticker:
            return instr.figi
    
    # ETF и остальное
    for instr in instruments_cache.etfs().instruments:
        if instr.ticker.upper() == ticker:
            return instr.figi
    
    raise ValueError(f"Тикер {ticker} не найден в кэше")

# Основная функция — с твоих мечт
async def get_5min(ticker: str) -> pd.DataFrame:
    ticker = ticker.upper()
    year = datetime.now().year
    filepath = DATA_DIR / ticker / f"5min_{year}.parquet"
    filepath.parent.mkdir(parents=True, exist_ok=True)

    # 1. Читаем кэш свечей
    if filepath.exists():
        df = pd.read_parquet(filepath)
        last_ts = df.index.max()
        from_dt = last_ts + timedelta(minutes=5)
        print(f"{ticker}: кэш до {last_ts.strftime('%Y-%m-%d %H:%M')}")
    else:
        df = pd.DataFrame()
        from_dt = datetime(year, 1, 1, tzinfo=timezone.utc)
        print(f"{ticker}: кэша нет → скачиваем с начала года")

    now_dt = datetime.now(timezone.utc)
    if from_dt >= now_dt:
        print(f"{ticker}: данные актуальны")
        return df.sort_index()

    # 2. Получаем FIGI через официальный кэш
    figi = get_figi(ticker)
    print(f"{ticker} → figi")

    # 3. Докачиваем свечи через лучший метод
    new_rows = []
    new_index = []

    async with AsyncClient(TOKEN) as client:
        async for candle in client.get_all_candles(
            figi=figi,
            from_=from_dt,
            to=now_dt,
            interval=CandleInterval.CANDLE_INTERVAL_5_MIN,
        ):
            new_index.append(candle.time)
            new_rows.append({
                'Open':  candle.open.units  + candle.open.nano  / 1e9,
                'High':  candle.high.units  + candle.high.nano  / 1e9,
                'Low':   candle.low.units   + candle.low.nano   / 1e9,
                'Close': candle.close.units + candle.close.nano / 1e9,
                'Volume': candle.volume,
            })

    if new_rows:
        new_df = pd.DataFrame(new_rows, index=new_index)
        new_df.index.name = "DateTime"
        df = pd.concat([df, new_df])
        df = df[~df.index.duplicated(keep='last')].sort_index()
        df.to_parquet(filepath, compression="zstd")
        print(f"{ticker}: обновлено! → {len(df):,} строк")
    else:
        print(f"{ticker}: новых данных нет")

    return df.sort_index()

# =================================================================
# Как использовать (в Jupyter/Colab/скрипте):
# =================================================================
# 1. Один раз при старте:
await init_cache()

# 2. Потом — сколько угодно:
df_lkoh  = await get_5min("LKOH")
df_sber  = await get_5min("SBER")    # → теперь точно BBG004730RP0!
df_imoex = await get_5min("IMOEX")   # → работает!
df_fx    = await get_5min("FXIT")

Получить OHLCV по тикеру intraday

In [37]:
def get_candles_5min(ticker: str,
                     start_date: str,
                     end_date: str = None,
                     token: str = TOKEN,
                     chunk_days: int = 7,
                     interval: CandleInterval = CandleInterval.CANDLE_INTERVAL_5_MIN) -> pd.DataFrame:
    """
    Скачивает 5-минутные свечи по ЛЮБОМУ тикеру (акции, индексы, ETF и т.д.)
    Просто пиши: get_candles_5min("LKOH", "2025-11-01")
                 get_candles_5min("IMOEX", "2025-01-01")
                 get_candles_5min("FXIT", "2025-06-01")
    """
    # 1. Находим FIGI автоматически
    print(f"Ищем FIGI для тикера '{ticker}'...")
    # figi = ticker_to_figi(ticker, token)
    figi = 'BBG004730N88'
    
    # 2. Даты
    if end_date is None:
        end_dt = datetime.now(timezone.utc).replace(hour=23, minute=59, second=59, microsecond=999999)
    else:
        end_dt = datetime.fromisoformat(f"{end_date}T23:59:59+00:00")
    
    start_dt = datetime.fromisoformat(f"{start_date}T00:00:00+00:00")
    all_rows = []
    current_start = start_dt

    print(f"Скачиваем {interval.name.split('_')[-1]} свечи с {start_dt.date()} по {end_dt.date()}...\n")

    while current_start < end_dt:
        current_end = min(current_start + timedelta(days=chunk_days), end_dt)
        print(f"  → {current_start.date()} → {current_end.date()}")

        try:
            with Client(token) as client:
                r = client.market_data.get_candles(
                    figi=figi,
                    from_=current_start,
                    to=current_end,
                    interval=interval
                )
        except Exception as e:
            print(f"  Ошибка API: {e}")
            current_start = current_end
            continue

        if not r.candles:
            print("  Нет свечей в этом куске")
        else:
            for candle in r.candles:
                ts = candle.time
                all_rows.append({
                    'DateTime': ts,
                    'Open':  candle.open.units + candle.open.nano / 1_000_000_000,
                    'High':  candle.high.units + candle.high.nano / 1_000_000_000,
                    'Low':   candle.low.units + candle.low.nano / 1_000_000_000,
                    'Close': candle.close.units + candle.close.nano / 1_000_000_000,
                    'Volume': candle.volume
                })
            print(f"  Получено {len(r.candles)} свечей")

        current_start = current_end + timedelta(seconds=1)

    if not all_rows:
        print("Нет данных за период")
        return pd.DataFrame()

    df = pd.DataFrame(all_rows)
    df.set_index('DateTime', inplace=True)
    df = df[~df.index.duplicated(keep='first')].sort_index()

    print(f"\nГОТОВО! Скачано {len(df):,} свечей по тикеру '{ticker}'")
    return df

In [38]:
get_candles_5min("SBER", "2025-11-01")

Ищем FIGI для тикера 'SBER'...
Скачиваем MIN свечи с 2025-11-01 по 2025-11-23...

  → 2025-11-01 → 2025-11-08
  Получено 1536 свечей
  → 2025-11-08 → 2025-11-15
  Получено 1517 свечей
  → 2025-11-15 → 2025-11-22
  Получено 1517 свечей
  → 2025-11-22 → 2025-11-23
  Получено 467 свечей

ГОТОВО! Скачано 5,035 свечей по тикеру 'SBER'


,Open,High,Low,Close,Volume
DateTime,,,,,
2025-11-01 03:55:00+00:00,292.00,292.00,292.00,292.00,1068
2025-11-01 04:00:00+00:00,292.02,292.48,292.00,292.29,46624
2025-11-01 04:05:00+00:00,292.30,292.38,292.16,292.38,23292
2025-11-01 04:10:00+00:00,292.38,292.38,290.35,290.76,178580
2025-11-01 04:15:00+00:00,290.76,291.61,290.76,291.55,46230
...,...,...,...,...,...
2025-11-23 16:40:00+00:00,309.12,309.22,308.79,308.99,1885
2025-11-23 16:45:00+00:00,308.99,309.00,308.70,308.71,3201
2025-11-23 16:50:00+00:00,308.71,309.18,308.71,309.11,1037
